# Import Dependencies

In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00


In [8]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.8 MB/s eta 0:00:00


In [27]:
from google_play_scraper import app, reviews, Sort, reviews_all
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import csv
import requests
from io import StringIO
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('punkt_tab')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Scraping Play Store Review **Honkai Impact 3rd**

In [4]:
# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreview = reviews_all(
    'com.miHoYo.bh3oversea',          # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=1000             # Jumlah maksimum ulasan yang ingin diambil
)


In [5]:
app_reviews_df = pd.DataFrame(scrapreview)

In [7]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,66a853be-f959-42ca-88d6-a2784de3cd62,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Ui terlalu berat, gamenya framedrop semenjak u...",1,530,7.9.0,2024-12-11 13:22:53,"Hai Kapten,\nTerima kasih atas kritik dan sara...",2024-03-20 07:45:28,7.9.0
1,b9a94c49-c341-4dff-a307-53720a98e526,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game yang bagus, terimakasih juga untuk perbai...",5,363,8.0.0,2025-01-23 04:58:42,None,NaT,8.0.0
2,fc2f18c3-1bdf-40fe-9825-e0383e845599,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kekurangan : 1.) Minimnya karakter cowok playa...,5,152,7.9.0,2024-12-05 05:20:31,None,NaT,7.9.0
3,72fb85b0-0c00-4baa-82b5-844b442f7773,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game nya bagus, tapi saya mengalami bug yang s...",5,8,8.0.0,2025-01-29 05:11:55,None,NaT,8.0.0
4,68d1de24-e137-40c2-a75f-c798b095e3b0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sebelumnya waktu jaman Mihoyo, sudah pernah ma...",5,26,7.9.0,2025-01-07 11:26:42,None,NaT,7.9.0


In [10]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65849 entries, 0 to 65848
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              65849 non-null  object        
 1   userName              65849 non-null  object        
 2   userImage             65849 non-null  object        
 3   content               65849 non-null  object        
 4   score                 65849 non-null  int64         
 5   thumbsUpCount         65849 non-null  int64         
 6   reviewCreatedVersion  39066 non-null  object        
 7   at                    65849 non-null  datetime64[ns]
 8   replyContent          14918 non-null  object        
 9   repliedAt             14918 non-null  datetime64[ns]
 10  appVersion            39066 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 5.5+ MB


# Cleaning Text

In [11]:
app_reviews_df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,26783
at,0
replyContent,50931
repliedAt,50931


In [12]:
clean_df = app_reviews_df.dropna(subset=['content'])

In [13]:
clean_df = clean_df.drop_duplicates()

In [16]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [20]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_casefoldingText'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [21]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_tokenizingText,text_stopword,text_akhir
0,66a853be-f959-42ca-88d6-a2784de3cd62,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Ui terlalu berat, gamenya framedrop semenjak u...",1,530,7.9.0,2024-12-11 13:22:53,"Hai Kapten,\nTerima kasih atas kritik dan sara...",2024-03-20 07:45:28,7.9.0,Ui terlalu berat gamenya framedrop semenjak up...,ui terlalu berat gamenya framedrop semenjak up...,"[ui, terlalu, berat, gamenya, framedrop, semen...","[ui, berat, gamenya, framedrop, semenjak, upda...",ui berat gamenya framedrop semenjak update par...
1,b9a94c49-c341-4dff-a307-53720a98e526,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game yang bagus, terimakasih juga untuk perbai...",5,363,8.0.0,2025-01-23 04:58:42,None,NaT,8.0.0,Game yang bagus terimakasih juga untuk perbaik...,game yang bagus terimakasih juga untuk perbaik...,"[game, yang, bagus, terimakasih, juga, untuk, ...","[game, bagus, terimakasih, perbaikan, optimali...",game bagus terimakasih perbaikan optimalisasin...
2,fc2f18c3-1bdf-40fe-9825-e0383e845599,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kekurangan : 1.) Minimnya karakter cowok playa...,5,152,7.9.0,2024-12-05 05:20:31,None,NaT,7.9.0,Kekurangan Minimnya karakter cowok playablec...,kekurangan minimnya karakter cowok playablec...,"[kekurangan, minimnya, karakter, cowok, playab...","[kekurangan, minimnya, karakter, cowok, playab...",kekurangan minimnya karakter cowok playablecum...
3,72fb85b0-0c00-4baa-82b5-844b442f7773,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game nya bagus, tapi saya mengalami bug yang s...",5,8,8.0.0,2025-01-29 05:11:55,None,NaT,8.0.0,Game nya bagus tapi saya mengalami bug yang sa...,game nya bagus tapi saya mengalami bug yang sa...,"[game, nya, bagus, tapi, saya, mengalami, bug,...","[game, bagus, mengalami, bug, mengganggu, dima...",game bagus mengalami bug mengganggu dimana kar...
4,68d1de24-e137-40c2-a75f-c798b095e3b0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sebelumnya waktu jaman Mihoyo, sudah pernah ma...",5,26,7.9.0,2025-01-07 11:26:42,None,NaT,7.9.0,Sebelumnya waktu jaman Mihoyo sudah pernah mai...,sebelumnya waktu jaman mihoyo sudah pernah mai...,"[sebelumnya, waktu, jaman, mihoyo, sudah, pern...","[jaman, mihoyo, main, level, valkyrie, udah, p...",jaman mihoyo main level valkyrie udah pensi de...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65844,b846f3a5-56b1-44ad-b399-38ea9e755b60,Ayunda Maharani,https://play-lh.googleusercontent.com/a/ACg8oc...,?????????????? ???????????????????????????????...,5,0,None,2022-06-04 12:18:33,None,NaT,None,Apa hah,apa hah,"[apa, hah]",[hah],hah
65845,5c0d3a77-9f4a-4f65-8df2-3cff5457ebdd,Farid Maulanayusuf,https://play-lh.googleusercontent.com/a-/ALV-U...,🤬,2,0,None,2024-05-24 07:48:52,"Hai Kapten,\nMaaf atas ketidaknyamananya. Moho...",2024-05-29 10:37:00,None,,,[],[],
65846,307c06dd-5de4-4f55-88e0-d16a19172ce7,Dhee 10,https://play-lh.googleusercontent.com/a/ACg8oc...,😉,2,0,None,2020-07-14 16:18:47,"Hi Captain, thank you for your support. If you...",2020-07-15 01:26:36,None,,,[],[],
65847,b0b1111f-462b-4322-9162-d0d440c04e4e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,😠,2,0,None,2018-12-07 07:50:25,None,NaT,None,,,[],[],


In [23]:
#terdapat beberapa teks yg kosong akibat pembersihan

#Mengubah teks kosong menjadi null
clean_df[['text_akhir']] = clean_df[['text_akhir']].applymap(lambda x: np.nan if isinstance(x, str) and x.strip() == "" else x)

#Mengubah teks yang hanya berupa 1 huruf menjadi null
clean_df[['text_akhir']] = clean_df[['text_akhir']].applymap(
    lambda x: np.nan if isinstance(x, str) and (len(x.strip()) <= 1) else x
)

<ipython-input-23-72debb96500b>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clean_df[['text_akhir']] = clean_df[['text_akhir']].applymap(lambda x: np.nan if isinstance(x, str) and x.strip() == "" else x)
<ipython-input-23-72debb96500b>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clean_df[['text_akhir']] = clean_df[['text_akhir']].applymap(


In [24]:
clean_df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,26783
at,0
replyContent,50931
repliedAt,50931


In [25]:
# Terdapat beberapa nilai null pada kolom text_akhir
clean_df = clean_df.dropna(subset=['text_akhir'])

In [26]:
clean_df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,26172
at,0
replyContent,49769
repliedAt,49769


# Pelabelan Data

In [28]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [29]:
# Fungsi untuk menentukan polaritas sentimen dari text

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score > 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
    else:
        polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [30]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    27703
negative    20588
neutral     16271
Name: count, dtype: int64


In [33]:
clean_df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,26172
at,0
replyContent,49769
repliedAt,49769


In [39]:
#Memilih kolom yang dipakai
clean_df = clean_df[['text_akhir', 'polarity_score', 'polarity']]

In [40]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64562 entries, 0 to 65844
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text_akhir      64562 non-null  object
 1   polarity_score  64562 non-null  int64 
 2   polarity        64562 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [41]:
#Tidak ada nilai null pada text sehingga aman untuk disimpan
clean_df.to_csv('clean_review_honkai.csv', index=False)